In [2]:
def TrouveCommunaute(communautes, i):
    ''' Renvoie le numero de la communaute du noeud i'''
    
    return communautes[i]

In [3]:
def QuiDansCommunaute(communautes, C):
    ''' Renvoie une liste des noeuds appartenant la communauté C'''

    commu = []

    for n in range(len(communautes)):
        if communautes[n] == C:
            commu.append(n)
    
    return commu

In [4]:
def TrouveVoisins(dico, i):
    ''' Renvoie une liste des voisins du sommet i'''
    
    l_res = []
    v = dico[i]

    for j in v:
        l_res.append(j[0])

    return l_res

In [5]:
def AjoutDansCommunaute(communautes, i, C):
    ''' Ajoute i à la communauté C dans la liste communautés '''
    # communautes est une liste où chaque élément d'indice i est le numéro de la communauté de i

    communautes[i] = C
    return communautes

In [6]:
def renommeCommu(commu):
    set_commu = list(set(commu))

    li_correspondance = [i for i in range(len(set_commu))]

    for i in range(len(commu)):
        for j in range(len(set_commu)):
            if commu[i] == set_commu[j]:
                commu[i] = li_correspondance[j]
    return commu

## Fonctions pour calcul du gain de modularité

In [7]:
def Calcul_m(dico):
    ''' Calcule la somme des poids du graphe '''
    
    somme_totale = 0
    somme_intermediaire = 0

    for sommet, voisins in dico.items():
        for v in voisins:
            if sommet != v[0]:                      # si ce n'est pas une boucle => on la comptera deux fois
                somme_intermediaire += v[1]
            else:                                   # si c'est une boucle
                somme_totale += v[1]

    somme_totale += somme_intermediaire//2

    return somme_totale      


In [8]:
def Calcul_ki(dico, i):
    ''' Calcule la somme des poids des arêtes du noeud i'''

    ki = 0

    for v in dico[i]:
        ki += v[1]

    return ki

In [9]:
def Calcul_S_tot(dico, noeuds_dans_commu, i):
    ''' Calcule la somme des poids des arêtes de la communauté testée '''
    # noeuds_dans_commu est une liste contenant les numéros des noeuds qui sont dans cette communauté
    # i est le noeud testé

    S_tot = 0

    for n in noeuds_dans_commu:
        if n != i:
            S_tot += Calcul_ki(dico, n)

    return S_tot

In [10]:
def Calcul_ki_in(dico, communautes, i, C):
    ''' Calcule la somme des poids des arêtes de i vers la communauté C'''

    ki_in = 0
    voisins = dico[i]

    for v in voisins:
        if communautes[v[0]] == C:
            ki_in += v[1]
    
    return ki_in


## Algorithme de Louvain

In [11]:
def Louvain_P1(dico):
    
    flag_modif = True
    m = Calcul_m(dico)

    # Initialisation de la liste communautes : chaque noeud est dans une communauté différente 
    communautes = [k for k in range(len(dico))]

    while flag_modif == True:       # tant qu'on modifie encore les communautés
        flag_modif = False

        for noeud in range(len(communautes)):       # pour chaque noeud

            liste_voisins = TrouveVoisins(dico, noeud)
            max = -1
            max_v = -1
            
            liste_commu_voisins = []
            for vo in liste_voisins:            # pour chaque voisin
                liste_commu_voisins.append(communautes[vo])         # ajoute dans liste_commu_voisins le numéro de la communauté de ce voisin
            
            # Calcul de ki
            ki = Calcul_ki(dico, noeud)

            for v in liste_voisins:     # pour chaque voisin du noeud

                if v != noeud:
                    n_commu_v = QuiDansCommunaute(communautes, communautes[v])

                    # Calcul de ki_in
                    ki_in = Calcul_ki_in(dico, communautes, noeud, communautes[v])

                    # Calcul de S tot
                    S_tot = Calcul_S_tot(dico, n_commu_v, noeud)

                    # Calcul du gain
                    gain = (1/(2*m))*((2*ki_in) - (S_tot * ki)/m)

                    if gain > max:      # si c'est un meilleur gain
                        max = gain
                        max_v = v
            
            if (communautes[noeud] != communautes[max_v]) and (max > 0):        # si on doit mettre le noeud dans une communauté différente
                AjoutDansCommunaute(communautes, noeud, communautes[max_v])     # on le met dans cette communauté en question
                flag_modif = True       # on a modifié

    return renommeCommu(communautes)
            

In [12]:
def Louvain_P2(dico_graphe, communautes):

    nouveau_graphe = {}
    nb_commu = len(set(communautes))

    for elt in range(nb_commu):
        li_sommet = []                          #récupère les sommets d'une communauté
        for sommet in range(len(communautes)):
            if communautes[sommet] == elt :
                li_sommet.append(sommet)

        poids_sommet_commu = 0                  #variable qui va contenir le poids du sommet (somme des aretes au sein de la commu)
        dico_arete_autre_commu = {}             #dico qui va contenir le nb d'aretes d'une commu vers chaque autres commu qui va correspondre au poids de l'arete reliant l'autre commu
        
        for sommet_commu in li_sommet :
            for arete in dico_graphe[sommet_commu]: #parcours les sommets de la commu en cours de traitement
                if arete[0] in li_sommet :       #parcours les arete de chaque sommet de la commu
                    poids_sommet_commu += arete[1]  #si le sommet relié au sommet en cours de traitement est aussi dans la commu on ajoute le poids de l'arete dans notre variable
                else:
                    if communautes[arete[0]] not in dico_arete_autre_commu :    #si il n'est pas dans la commu
                        dico_arete_autre_commu[communautes[arete[0]]] = arete[1] #si c'est le premier/seul elt connecté à notre commu on ajoute dans le dico la commu associé à son poids
                    else :
                        dico_arete_autre_commu[communautes[arete[0]]] += arete[1] #sinon on additionne le poids de la nouvelle arete reliant la commu
        
        for commu, poids in dico_arete_autre_commu.items():     #partie permettant de creer le nouveau graphe
            if elt not in nouveau_graphe :                      #si on n'a pas encore traité la commu
                nouveau_graphe[elt]=[(elt, poids_sommet_commu)] #on ajoute l'arete qui va sur elle meme permettant d'indiquer le poids du sommet
                nouveau_graphe[elt].append((commu, poids))      #on ajoute ensuite les aretes vers l'autre commu en cours de traitement
            else : 
                nouveau_graphe[elt].append((commu, poids))      #on a déjà la commu dans le nouveau graphe, ça veut donc dire qu'on traite les commu suivante vers laquelle il est relié

    return nouveau_graphe

In [23]:
def Louvain_Final_jusqua2(dico):
    ''' Appelle les 2 parties de l'algorithme de Louvain itérativement'''

    nouveau_graphe = dico
    historique_graphes = [nouveau_graphe]

    while len(nouveau_graphe) > 2:
        nvelles_communautes = Louvain_P1(nouveau_graphe)
        nouveau_graphe = Louvain_P2(nouveau_graphe, nvelles_communautes)
        historique_graphes.append(nouveau_graphe)

    return historique_graphes
    

In [24]:
def Louvain_Final_jusqua_pas_diminution(dico):
    ''' Appelle les 2 parties de l'algorithme de Louvain itérativement'''

    nouveau_graphe = dico
    historique_graphes = [nouveau_graphe]

    while True:
        
        nvelles_communautes = Louvain_P1(nouveau_graphe)
        nouveau_graphe = Louvain_P2(nouveau_graphe, nvelles_communautes)
        
        if len(nouveau_graphe) == len(historique_graphes[-1]):
            break

        historique_graphes.append(nouveau_graphe)

    return historique_graphes

## Tests !

In [15]:
# Le graphe test contient 21 sommets et 34 arêtes

dico_test = {0:[(1,1)],
            1:[(0,1), (2,1)],
            2:[(1,1), (3,1), (4,1)],
            3:[(2,1), (4,1), (8,1)],
            4:[(2,1), (3,1), (5,1), (7,1)],
            5:[(4,1), (6,1), (17,1)],
            6:[(5,1), (7,1), (19,1)],
            7:[(4,1), (6,1), (9,1)],
            8:[(3,1)],
            9:[(7,1), (10,1), (11,1)],
            10:[(9,1), (11,1)],
            11:[(9,1), (10,1), (12,1)],
            12:[(11,1), (14,1), (17,1), (19,1)],
            13:[(14,1), (15,1), (16,1), (19,1)],
            14:[(12,1), (13,1), (15,1)],
            15:[(13,1), (14,1), (16,1), (17,1)],
            16:[(13,1), (15,1), (17,1), (18,1), (20,1)],
            17:[(5,1), (12,1), (15,1), (16,1), (18,1)],
            18:[(16,1), (17,1), (19,1), (20,1)],
            19:[(6,1), (12,1), (13,1), (18,1), (20,1)],
            20:[(16,1), (18,1), (19,1)]
            }

In [16]:
dico_test2 = {0:[(1,1), (2,1), (3,1)],
              1:[(0,1), (2,1), (3,1)],
              2:[(0,1), (1,1), (3,1)],
              3:[(0,1), (1,1), (2,1), (4,1)],
              4:[(5,1), (6,1), (7,1), (3,1)],
              5:[(4,1), (6,1), (7,1)],
              6:[(4,1), (5,1), (7,1)],
              7:[(4,1), (5,1), (6,1)],}

In [17]:
dico_SV = {0:[(2,1), (3,1), (4,1), (5,1)],
            1:[(2,1), (4,1), (7,1)],
            2:[(0,1), (1,1), (4,1), (5,1), (6,1),],
            3:[(0,1), (7,1)],
            4:[(0,1), (1,1), (2,1), (10,1)],
            5:[(0,1), (2,1), (7,1), (11,1)],
            6:[(2,1), (7,1), (11,1)],
            7:[(1,1), (3,1), (5,1), (6,1)],
            8:[(9,1), (10,1), (11,1), (14,1), (15,1)],
            9:[(8,1), (12,1), (14,1)],
            10:[(4,1), (8,1), (11,1), (12,1), (13,1), (14,1)],
            11:[(5,1), (6,1), (8,1), (10,1), (13,1)],
            12:[(9,1), (10,1)],
            13:[(10,1), (11,1)],
            14:[(8,1), (9,1), (10,1)],
            15:[(8,1)]}

In [25]:
res = Louvain_Final_jusqua2(dico_SV)
res2 = Louvain_Final_jusqua_pas_diminution(dico_SV)
print('test 1')
for d in res:
    print(d)
print('test 2')
for d in res2:
    print(d)
print('test 3 : ', res==res2)

test 1
{0: [(2, 1), (3, 1), (4, 1), (5, 1)], 1: [(2, 1), (4, 1), (7, 1)], 2: [(0, 1), (1, 1), (4, 1), (5, 1), (6, 1)], 3: [(0, 1), (7, 1)], 4: [(0, 1), (1, 1), (2, 1), (10, 1)], 5: [(0, 1), (2, 1), (7, 1), (11, 1)], 6: [(2, 1), (7, 1), (11, 1)], 7: [(1, 1), (3, 1), (5, 1), (6, 1)], 8: [(9, 1), (10, 1), (11, 1), (14, 1), (15, 1)], 9: [(8, 1), (12, 1), (14, 1)], 10: [(4, 1), (8, 1), (11, 1), (12, 1), (13, 1), (14, 1)], 11: [(5, 1), (6, 1), (8, 1), (10, 1), (13, 1)], 12: [(9, 1), (10, 1)], 13: [(10, 1), (11, 1)], 14: [(8, 1), (9, 1), (10, 1)], 15: [(8, 1)]}
{0: [(0, 2), (1, 1), (3, 1), (2, 3)], 1: [(1, 14), (3, 4), (2, 1), (0, 1)], 2: [(2, 16), (0, 3), (1, 1)], 3: [(3, 4), (1, 4), (0, 1)]}
{0: [(0, 24), (1, 3)], 1: [(1, 26), (0, 3)]}
test 2
{0: [(2, 1), (3, 1), (4, 1), (5, 1)], 1: [(2, 1), (4, 1), (7, 1)], 2: [(0, 1), (1, 1), (4, 1), (5, 1), (6, 1)], 3: [(0, 1), (7, 1)], 4: [(0, 1), (1, 1), (2, 1), (10, 1)], 5: [(0, 1), (2, 1), (7, 1), (11, 1)], 6: [(2, 1), (7, 1), (11, 1)], 7: [(1, 1), (

In [19]:
dicoV2 = {0: [(0, 2), (1, 1), (3, 1), (2, 3)], 
          1: [(1, 14), (3, 4), (2, 1), (0, 1)], 
          2: [(2, 16), (0, 3), (1, 1)], 
          3: [(3, 4), (1, 4), (0, 1)]}

In [20]:
dicoV3 = {0: [(0,26), (1, 3)],
          1: [(1,24), (0,3)]}